Title
===

Mert Sami

Myrthe Peeters

# Abstract

# Research Question

Are female doctors in Houston College of Medicine being discriminated against in giving promotions and setting salaries compared to their male counterparts in 1994 and 1995?


# Data

In [15]:
import numpy as np

In [16]:
from scipy.stats import wilcoxon, mannwhitneyu

In [17]:
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd

df = pd.read_csv(https://www.kaggle.com/hjmjerry/gender-discrimination/data)

SyntaxError: invalid syntax (<ipython-input-17-c83ab564e089>, line 5)

In [65]:
import pandas as pd
df_lawsuit = pd.read_csv('Lawsuit.csv')
import matplotlib.pyplot as plt
%matplotlib inline

FileNotFoundError: File b'Lawsuit.csv' does not exist

In [18]:
id1 = [1, 1, 0, 0, 7.4, 9, 3, 77836, 84612]
id2 = [1, 1, 0, 0, 6.7, 10, 2, 69994, 78497]
id3 = [1, 1, 0, 0, 8.1, 6, 1, 82872, 67756]
id4 = [1, 1, 1, 1,  5.1, 27, 3, 155196, 173220]
id5 = [1, 1, 0, 0, 7, 10, 3, 89268, 96099]
id6 = [1, 1, 0, 1, 7.7 , 10, 3, 79714, 87531]
id7 = [1, 1, 0, 1, 7.3, 9, 2, 89781, 99972]
id8 = [1, 1, 1, 1, 5.3, 11, 3, 151423, 166601]
id9 = [1, 1, 0, 0, 7.7, 18, 3, 81271, 85437]
id10 = [1, 1, 0, 0, 7.5, 9, 2, 73018, 91369]
id11 = [1, 1, 0, 0, 7.8, 12, 2, 80214, 88676]
id12 = [1, 1, 0, 1, 7.3, 8, 2, 110302, 116654]
id13 = [1, 1, 1, 1, 5.4, 18, 3, 137106, 149222]
id14 = [1, 1, 0, 0, 7.5, 15, 2, 77545, 85041]
id15 = [1, 1, 0, 0, 6.8, 10, 2, 90223, 97640]
id16 = [1, 1, 0, 0, 8.1, 9, 3, 81119, 91800]
id17 = [1, 1, 0, 0, 7.3, 16, 3, 90178, 101828]
id18 = [1, 1, 0, 1, 7.7, 9, 3, 105564, 117743]
id19 = [1, 1, 0, 0, 7.1, 37, 2, 115804, 126989]
id20 = [1, 1, 0, 0, 7.2, 14, 3, 113149, 124432]
id21 = [1, 1, 0, 1, 7.6, 27, 3, 157510, 168673]
id22 = [1, 1, 0, 0, 7.1, 14, 3, 97104, 104090]
id23 = [1, 1, 0, 1, 7.3, 17, 3, 131815, 145843]
id24 = [1, 1, 0, 1, 6.9, 23, 3, 134135, 151837]
id25 = [1, 1, 0, 1, 7.4, 8, 3, 92198, 98155]
id26 = [1, 1, 0, 1, 6.9, 19, 3, 99304, 110321]
id27 = [1, 1, 0, 0, 8, 15, 2, 82132, 90058]
id28 = [1, 1, 0, 1, 7, 16, 3, 108770, 119697]
id29 = [1, 1, 0, 1, 6.9, 15, 3, 109775, 120208]
id30 = [1, 1, 0, 0, 7.7, 7, 1, 66375, 71139]
id31 = [1, 0, 0, 0, 7.2, 2, 1, 57838, 62907]
id32 = [1, 0, 0, 1, 7.7, 12, 2, 90631, 99079]
id33 = [1, 0, 0, 1, 7.9, 7, 1, 60930, 66190]
id34 = [1, 0, 0, 0, 7.1, 14, 2, 87231, 95385]
id35 = [1, 0, 0, 0, 7.4, 8, 2, 68582, 76987]
id36 = [1, 0, 0, 0, 7.3, 7, 1, 63280, 68420]
id37 = [1, 0, 0, 0, 7.9, 16, 3, 106412, 116959]
id38 = [1, 0, 0, 1, 7.2, 3, 1, 75337, 71066]
id39 = [1, 0, 1, 1, 7.4, 16, 3, 143334, 153524]
id40 = [1, 0, 0, 0, 8, 12, 3, 90771, 102927]
id41 = [1, 0, 0, 0, 7.1, 8, 2, 81769, 88896]
id42 = [1, 0, 0, 1, 7.5, 11, 1, 94103, 102983]
id43 = [1, 0, 0, 1, 7.6, 10, 1, 102631, 111125]
id44 = [1, 0, 1, 1, 6, 15, 1, 109164, 120696]
id45 = [1, 0, 0, 0, 7.1, 9, 1, 63779, 73033]
id46 = [1, 0, 0, 0, 8, 6, 1, 75362, 81615]
id47 = [1, 0, 1, 0, 4.6, 13, 2, 124836, 137791]
id48 = [1, 0, 0, 0, 7.7, 13, 1, 77012, 84536]
id49 = [1, 0, 0, 1, 7.3, 6, 1, 67277, 72721]
id50 = [1, 0, 0, 1, 7.1, 9, 1, 73448, 80294]
id51 = [2, 1, 1, 1, 5.1, 10, 1, 97707, 101617]
id52 = [2, 1, 1, 1, 5.1, 9, 3, 106130, 118258]
id53 = [2, 1, 0, 1, 6.8, 8, 3, 81049, 88505]
id54 = [2, 1, 1, 1, 5.5, 25, 3, 140315, 150710]
id55 = [2, 1, 0, 1, 7.1, 11, 3, 84571, 92879]
id56 = [2, 1, 0, 1, 6.6, 9, 2, 58126, 64522]
id57 = [2, 1, 1, 1, 6.1, 27, 3, 133284, 151373]
id58 = [2, 1, 0, 1, 7.5, 32, 2, 142333, 154372]
id59 = [2, 1, 0, 0, 7.3, 23, 3, 71073, 78715]
id60 = [2, 1, 0, 0, 7.5, 14, 2, 76255, 82466]
id61 = [2, 1, 0, 1, 7.8, 11, 3, 105348, 118635]
id62 = [2, 1, 0, 0, 7.5, 9, 3, 86844, 97061]
id63 = [2, 1, 0, 0, 7, 5, 2, 52582, 58923]
id64 = [2, 1, 0, 1, 7.5, 12, 3, 110836, 120397]
id65 = [2, 1, 1, 1, 5.7, 18, 3, 144150, 151887]
id66 = [2, 1, 1, 1, 6, 19, 3, 122240, 134882]
id67 = [2, 1, 0, 0, 7.1, 10, 3, 68573, 75286]
id68 = [2, 1, 0, 0, 7.9, 8, 2, 36946, 69526]
id69 = [2, 1, 1, 1, 5.9, 4, 1, 71404, 78589]
id70 = [2, 1, 0, 1, 7.6, 14, 3, 102076, 112009]
id71 = [2, 0, 1, 1, 5.3, 10, 2, 94715, 106608]
id72 = [2, 0, 0, 1, 8.1, 4, 1, 58634, 62601]
id73 = [2, 0, 0, 1, 7.9, 14, 3, 119851, 131050]
id74 = [2, 0, 1, 0, 5.5, 9, 3, 78651, 88661]
id75 = [2, 0, 0, 0, 6.9, 8, 2, 57862, 63728]
id76 = [2, 0, 1, 0, 4.9, 3, 1, 67113, 74272]
id77 = [2, 0, 1, 0, 5.3, 12, 3, 81078, 86221]
id78 = [2, 0, 0, 1, 8.5, 7, 1, 60403, 68828]
id79 = [2, 0, 0, 1, 7.6, 12, 2, 90964, 97303]
id80 = [2, 0, 0, 1, 8.4, 4, 1, 12945, 57841]
id81 = [2, 0, 0, 0, 7.4, 5, 1, 56550, 62323]
id82 = [2, 0, 0, 0, 7.2, 2, 1, 34514, 38675]
id83 = [2, 0, 0, 0, 8, 8, 2, 56177, 61652]
id84 = [2, 0, 1, 0, 5, 2, 1, 50588, 57431]
id85 = [2, 0, 0, 1, 7.7, 5, 1, 60415, 66323]
id86 = [2, 0, 1, 0, 6.1, 11, 3, 91409, 102511]
id87 = [2, 0, 0, 1, 7.8, 3, 1, 56250, 59530]
id88 = [2, 0, 1, 1, 5.7, 16, 2, 127725, 143348]
id89 = [2, 0, 1, 0, 5.4, 5, 1, 63426, 71068]
id90 = [2, 0, 0, 0, 8, 5, 1, 45826, 48138]
id91 = [3, 1, 0, 1, 6.7, 24, 3, 154326, 165884]
id92 = [3, 1, 0, 0, 5.9, 11, 2, 87351, 95429]
id93 = [3, 1, 1, 1, 4.1, 5, 1, 123859, 130292]
id94 = [3, 1, 1, 0, 4.8, 16, 3, 118121, 130697]
id95 = [3, 1, 1, 1, 4.3, 11, 3, 174479, 193342]
id96 = [3, 1, 1, 1, 4.3, 16, 3, 196704, 218955]
id97 = [3, 1, 1, 1, 4.3, 16, 3, 186953, 206984]
id98 = [3, 1, 1, 0, 4.2, 9, 2, 107862, 117240]
id99 = [3, 1, 1, 0, 3.9, 5, 1, 105821, 116932]
id100 = [3, 1, 1, 1, 5.6, 6, 1, 114730, 126118]
id101 = [3, 0, 1, 0, 3.9, 9, 3, 142382, 151698]
id102 = [3, 0, 0, 0, 7.3, 9, 1, 67581, 72896]
id103 = [3, 0, 0, 0, 6.8, 6, 1, 67934, 75659]
id104 = [3, 0, 1, 0, 5, 10, 3, 118101, 129862]
id105 = [3, 0, 1, 1, 5.2, 9, 2, 127710, 143799]
id106 = [3, 0, 0, 0, 6.5, 5, 1, 70794, 77113]
id107 = [3, 0, 1, 0, 5.7, 4, 1, 81728, 91764]
id108 = [3, 0, 0, 0, 7, 7, 1, 71015, 80747]
id109 = [3, 0, 0, 1, 7.3, 11, 2, 117276, 126139]
id110 = [3, 0, 0, 0, 7.2, 10, 2, 106535, 117574]
id111 = [3, 0, 0, 0, 8.4, 5, 1, 68825, 74209]
id112 = [4, 1, 1, 1, 3.1, 6, 1, 109599, 121829]
id113 = [4, 1, 0, 1, 5.2, 17, 3, 154159, 169191]
id114 = [4, 1, 1, 1, 3.3, 10, 2, 145503, 162253]
id115 = [4, 1, 1, 1, 3.2, 15, 2, 161383, 181196]
id116 = [4, 1, 0, 1, 4.8, 10, 3, 113443, 126189]
id117 = [4, 1, 1, 1, 2.1, 5, 1, 118561, 131704]
id118 = [4, 1, 1, 1, 2.9, 8, 2, 147709, 161814]
id119 = [4, 1, 1, 1, 3.1, 7, 2, 121969, 136415]
id120 = [4, 1, 0, 0, 5.2, 11, 2, 101890, 113893]
id121 = [4, 1, 1, 1, 4, 7, 1, 114233, 127955]
id122 = [4, 0, 1, 1, 4.2, 4, 1, 77087, 84890]
id123 = [4, 0, 0, 1, 5.5, 6, 1, 113382, 124153]
id124 = [4, 0, 1, 1, 2.7, 4, 1, 102177, 112844]
id125 = [4, 0, 1, 1, 3.1, 2, 2, 116506, 127815]
id126 = [4, 0, 1, 1, 3.1, 5, 1, 130286, 145073]
id127 = [4, 0, 1, 1, 3.6, 16, 3, 159687, 177706]
id128 = [4, 0, 1, 1, 3.4, 2, 1, 107750, 117214]
id129 = [4, 0, 0, 1, 5.3, 4, 1, 91211, 103513]
id130 = [4, 0, 1, 1, 3, 6, 2, 135216, 150489]
id131 = [4, 0, 1, 1, 3.2, 10, 2, 164048, 182049]
id132 = [4, 0, 1, 1, 3.8, 5, 1, 84390, 92512]
id133 = [4, 0, 1, 1, 3.1, 14, 3, 170428, 186726]
id134 = [4, 0, 1, 1, 2.6, 5, 1, 124994, 140026]
id135 = [4, 0, 0, 1, 5.7, 13, 1, 90130, 95908]
id136 = [4, 0, 0, 1, 6.3, 5, 1, 102021, 109758]
id137 = [4, 0, 1, 0, 4, 4, 2, 88484, 99505]
id138 = [4, 0, 1, 1, 3, 7, 1, 110975, 123139]
id139 = [4, 0, 0, 1, 5.5, 3, 1, 81422, 89202]
id140 = [4, 0, 1, 1, 3.1, 10, 1, 130939, 142566]
id141 = [4, 0, 1, 1, 3.7, 1, 1, 111630, 125268]
id142 = [5, 1, 1, 1, 3.7, 11, 1, 182945, 198762]
id143 = [5, 1, 1, 0, 3.2, 13, 2, 163671, 181774]
id144 = [5, 1, 1, 1, 4.3, 9, 2, 204433, 226581]
id145 = [5, 1, 1, 1, 3.2, 4, 1, 135139, 148117]
id146 = [5, 1, 0, 1, 5.6, 11, 3, 147213, 155826]
id147 = [5, 1, 1, 1, 4, 13, 2, 207737, 229706]
id148 = [5, 1, 1, 1, 3.1, 18, 3, 244025, 268210]
id149 = [5, 1, 0, 1, 5.2, 12, 3, 167199, 182401]
id150 = [5, 1, 1, 1, 4, 14, 3, 209377, 225498]
id151 = [5, 1, 1, 1, 3.7, 10, 2, 209189, 230270]
id152 = [5, 1, 1, 1, 3.2, 11, 2, 186973, 204769]
id153 = [5, 1, 1, 1, 3.1, 10, 3, 209171, 229301]
id154 = [5, 1, 1, 1, 4.5, 15, 3, 230268, 253090]
id155 = [5, 1, 1, 1, 4.4, 8, 2, 186022, 207634]
id156 = [5, 1, 1, 1, 4.2, 7, 1, 179897, 196338]
id157 = [5, 1, 1, 1, 3.6, 12, 2, 236470, 265121]
id158 = [5, 1, 0, 0, 5.7, 15, 3, 172793, 184391]
id159 = [5, 1, 1, 1, 3.7, 13, 3, 285301, 297397]
id160 = [5, 1, 1, 1, 3.6, 8, 3, 200543, 219584]
id161 = [5, 1, 1, 1, 4.8, 6, 1, 152485, 1668170]
id162 = [5, 1, 1, 1, 4, 5, 1, 154644, 170967]
id163 = [5, 1, 1, 1, 3.1, 22, 3, 255654, 285939]
id164 = [5, 1, 1, 1, 3.5, 2, 1, 138896, 152284]
id165 = [5, 1, 1, 1, 3.1, 18, 3, 260631, 284841]
id166 = [5, 1, 1, 1, 3, 4, 1, 183972, 201691]
id167 = [5, 1, 1, 1, 3, 5, 1, 128027, 142963]
id168 = [5, 1, 1, 1, 4, 6, 1, 161815, 173837]
id169 = [5, 1, 0, 1, 5.9, 6, 1, 149011, 163795]
id170 = [5, 1, 1, 1, 3.3, 8, 2, 167305, 183834]
id171 = [5, 1, 1, 1, 2.7, 5, 1, 153549, 168707]
id172 = [5, 1, 1, 1, 3.4, 37, 3, 300975, 332119]
id173 = [5, 1, 1, 0, 2.4, 3, 1, 124442, 131361]
id174 = [5, 1, 1, 1, 3.8, 14, 2, 163486, 175271]
id175 = [5, 1, 1, 1, 3.2, 7, 1, 155006, 169118]
id176 = [5, 1, 0, 0, 5.1, 8, 3, 129499, 141661]
id177 = [5, 1, 1, 1, 3.5, 14, 2, 216391, 234992]
id178 = [5, 1, 1, 0, 2.9, 8, 2, 171016, 189308]
id179 = [5, 1, 1, 1, 3.5, 25, 3, 246763, 279255]
id180 = [5, 1, 1, 1, 3.4, 5, 1, 175396, 189904]
id181 = [5, 1, 0, 1, 5.1, 8, 2, 144542, 158513]
id182 = [5, 1, 0, 1, 4.7, 21, 3, 275377, 294336]
id183 = [5, 1, 1, 1, 4, 18, 3, 262090, 289778]
id184 = [5, 1, 0, 0, 5.1, 2, 1, 250000, 276163]
id185 = [5, 1, 0, 1, 4.6, 10, 2, 158243, 177587]
id186 = [5, 1, 1, 0, 3.6, 26, 3, 218341, 245432]
id187 = [5, 1, 1, 0, 3.3, 5, 1, 141044, 157509]
id188 = [5, 1, 1, 0, 3.9, 10, 2, 164174, 183538]
id189 = [5, 1, 1, 1, 3.3, 16, 3, 203152, 216873]
id190 = [5, 1, 1, 1, 3.6, 8, 1, 194784, 217478]
id191 = [5, 1, 1, 1, 3, 15, 3, 223040, 239253]
id192 = [5, 0, 1, 0, 4.2, 3, 1, 138635, 149539]
id193 = [5, 0, 1, 1, 4.1, 5, 1, 176028, 192799]
id194 = [5, 0, 1, 1, 3.9, 2, 21, 126343, 136988]
id195 = [5, 0, 1, 1, 3.6, 14, 3, 249687, 267554]
id196 = [5, 0, 1, 1, 3.8, 5, 1, 127525, 139017]
id197 = [5, 0, 1, 1, 3.1, 13, 3, 281058, 313400]
id198 = [5, 0, 1, 1, 3.6, 4, 1, 149631, 165420]
id199 = [5, 0, 1, 1, 3.8, 4, 1, 142249, 154386]
id200 = [5, 0, 1, 1, 4.7, 7, 1, 166863, 189852]
id201 = [5, 0, 1, 1, 2.9, 7, 1, 164080, 180937]
id202 = [5, 0, 1, 0, 2.9, 1, 1, 112221, 126149]
id203 = [5, 0, 1, 0, 3.8, 5, 1, 161352, 177997]
id204 = [5, 0, 1, 1, 2.8, 7, 1, 2.9818, 233774]
id205 = [5, 0, 0, 1, 5.1, 6, 1, 133097, 147327]
id206 = [5, 0, 1, 1, 3.7, 5, 1, 171391, 185895]
id207 = [5, 0, 1, 1, 3.7, 6, 1, 166482, 180608]
id208 = [5, 0, 1, 1, 4, 12, 1, 242614, 265856]
id209 = [5, 0, 1, 1, 3.8, 6, 1, 146396, 161031]
id210 = [5, 0, 1, 1, 3.8, 6, 1, 130153, 139887]
id211 = [5, 0, 1, 1, 3.7, 8, 2, 175791, 195678]
id212 = [5, 0, 1, 0, 3.4, 3, 1, 138775, 152231]
id213 = [5, 0, 1, 1, 4.3, 6, 1, 167844, 189487]
id214 = [5, 0, 1, 1, 3.9, 10, 2, 202231, 225599]
id215 = [5, 0, 1, 0, 3.9, 2, 1, 106802, 114961]
id216 = [5, 0, 0, 1, 5.8, 23, 3, 204279, 227790]
id217 = [5, 0, 1, 1, 4.5, 8, 3, 185413, 207462]
id218 = [5, 0, 0, 1, 4.6, 9, 2, 150258, 160253]
id219 = [5, 0, 1, 1, 4, 5, 1, 128433, 145718]
id220 = [5, 0, 0, 1, 5.4, 14, 1, 123940, 138566]
id221 = [5, 0, 0, 1, 5.4, 3, 1, 107524, 116283]
id222 = [6, 1, 1, 1, 2.7, 6, 1, 214219, 236305]
id223 = [6, 1, 1, 1, 2.1, 5, 1, 312966, 336960]
id224 = [6, 1, 1, 1, 1.9, 21, 2, 342834, 367284]
id225 = [6, 1, 1, 1, 2.1, 16, 3, 312892, 344801]
id226 = [6, 1, 1, 1, 2.5, 13, 3, 317399, 345204]
id227 = [6, 1, 1, 1, 2.4, 21, 3, 342662, 369712]
id228 = [6, 1, 1, 1, 2.5, 5, 2, 247953, 273737]
id229 = [6, 1, 1, 1, 3.2, 11, 3, 339533, 387205]
id230 = [6, 1, 1, 1, 2.4, 22, 3, 393408, 435572]
id231 = [6, 1, 1, 1, 2.5, 5, 1, 206097, 220899]
id232 = [6, 1, 1, 1, 2.4, 27, 3, 419129, 472589]
id233 = [6, 1, 1, 1, 1.9, 5, 1, 250667, 270417]
id234 = [6, 1, 1, 1, 2.9, 6, 1, 212780, 237094]
id235 = [6, 1, 1, 1, 2.7, 7, 1, 232735, 261236]
id236 = [6, 1, 0, 1, 4.4, 11, 2, 244940, 269554]
id237 = [6, 1, 1, 1, 2.2, 13, 2, 287433, 314926]
id238 = [6, 1, 1, 1, 2.8, 7, 1, 279661, 317179]
id239 = [6, 1, 1, 1, 2.7, 12, 3, 428876, 462342]
id240 = [6, 1, 1, 1, 2.4, 10, 2, 326460, 364139]
id241 = [6, 1, 1, 1, 3.2, 10, 3, 366150, 408898]
id242 = [6, 1, 1, 1, 2.93, 8, 3, 299582, 339637]
id243 = [6, 1, 1, 1, 2.1, 5, 2, 259576, 280168]
id244 = [6, 1, 1, 1, 1.9, 12, 1, 31675, 349294]
id245 = [6, 1, 1, 1, 2.5, 7, 1, 220374, 239614]
id246 = [6, 1, 1, 1, 1.3, 7, 1, 262328, 295619]
id247 = [6, 1, 1, 1, 2.5, 14, 1, 337885, 376651]
id248 = [6, 1, 1, 1, 2.9, 14, 2, 355982, 389994]
id249 = [6, 1, 1, 0, 3.1, 16, 2, 298359, 320315]
id250 = [6, 1, 1, 1, 3, 6, 1, 247576, 271773]
id251 = [6, 1, 1, 1, 3.3, 15, 1, 251079, 275246]
id252 = [6, 1, 1, 0, 2.4, 2, 1, 154310, 171664]
id253 = [6, 1, 1, 1, 3.2, 25, 3, 411219, 455468]
id254 = [6, 1, 1, 1, 3, 15, 1, 266880, 297902]
id255 = [6, 1, 1, 1, 2.9, 5, 1, 223736, 248014]
id256 = [6, 1, 1, 1, 2.7, 10, 3, 322698, 356961]
id257 = [6, 0, 1, 1, 2.9, 8, 2, 297174, 323699]
id258 = [6, 0, 1, 1, 2.6, 9, 3, 308081, 339664]
id259 = [6, 0, 1, 1, 2.8, 8, 1, 211269, 241577]
id260 = [6, 0, 1, 1, 2.4, 2, 1, 210801, 233905]
id261 = [6, 0, 1, 1, 3.2, 5, 1, 244551, 265502]

In [19]:
datawag = [[1, 1, 0, 0, 7.4, 9, 3, 77836, 84612], 
           [1, 1, 0, 0, 6.7, 10, 2, 69994, 78497], 
           [1, 1, 0, 0, 8.1, 6, 1, 82872, 67756], 
           [1, 1, 1, 1, 5.1, 27, 3, 155196, 173220],
           [1, 1, 0, 0, 7.0, 10, 3, 89268, 96099], 
           [1, 1, 0, 1, 7.7, 10, 3, 79714, 87531], 
           [1, 1, 0, 1, 7.3, 9, 2, 89781, 99972], 
           [1, 1, 1, 1, 5.3, 11, 3, 151423, 166601], 
           [1, 1, 0, 0, 7.7, 18, 3, 81271, 85437], 
           [1, 1, 0, 0, 7.5, 9.0, 2, 73018, 91369], 
           [1, 1, 0, 0, 7.8, 12, 2, 80214, 88676], 
           [1, 1, 0, 1, 7.3, 8.0, 2, 110302, 116654], 
           [1, 1, 1, 1, 5.4, 18, 3, 137106, 149222], 
           [1, 1, 0, 0, 7.5, 15, 2, 77545, 85041], 
           [1, 1, 0, 0, 6.8, 10, 2, 90223, 97640], 
           [1, 1, 0, 0, 8.1, 9.0, 3, 81119, 91800], 
           [1, 1, 0, 0, 7.3, 16, 3, 90178, 101828], 
           [1, 1, 0, 1, 7.7, 9.0, 3, 105564, 117743], 
           [1, 1, 0, 0, 7.1, 37, 2, 115804, 126989], 
           [1, 1, 0, 0, 7.2, 14, 3, 113149, 124432], 
           [1, 1, 0, 1, 7.6, 27, 3, 157510, 168673], 
           [1, 1, 0, 0, 7.1, 14, 3, 97104, 104090], 
           [1, 1, 0, 1, 7.3, 17, 3, 131815, 145843], 
           [1, 1, 0, 1, 6.9, 23, 3, 134135, 151837], 
           [1, 1, 0, 1, 7.4, 8, 3, 92198, 98155],  
           [1, 1, 0, 1, 6.9, 19, 3, 99304, 110321], 
           [1, 1, 0, 0, 8.0, 15, 2, 82132, 90058], 
           [1, 1, 0, 1, 7.0, 16, 3, 108770, 119697], 
           [1, 1, 0, 1, 6.9, 15, 3, 109775, 120208], 
           [1, 1, 0, 0, 7.7, 7, 1, 66375, 71139], 
           [1, 0, 0, 0, 7.2, 2, 1, 57838, 62907], 
           [1, 0, 0, 1, 7.7, 12, 2, 90631, 99079], 
           [1, 0, 0, 1, 7.9, 7, 1, 60930, 66190], 
           [1, 0, 0, 0, 7.1, 14, 2, 87231, 95385], 
           [1, 0, 0, 0, 7.4, 8, 2, 68582, 76987], 
           [1, 0, 0, 0, 7.3, 7, 1, 63280, 68420], 
           [1, 0, 0, 0, 7.9, 16, 3, 106412, 116959], 
           [1, 0, 0, 1, 7.2, 3, 1, 75337, 71066], 
           [1, 0, 1, 1, 7.4, 16, 3, 143334, 153524], 
           [1, 0, 0, 0, 8.0, 12, 3, 90771, 102927], 
           [1, 0, 0, 0, 7.1, 8, 2, 81769, 88896], 
           [1, 0, 0, 1, 7.5, 11, 1, 94103, 102983], 
           [1, 0, 0, 1, 7.6, 10, 1, 102631, 111125], 
           [1, 0, 1, 1, 6.0, 15, 1, 109164, 120696], 
           [1, 0, 0, 0, 7.1, 9, 1, 63779, 73033], 
           [1, 0, 0, 0, 8, 6, 1, 75362, 81615], 
           [1, 0, 1, 0, 4.6, 13, 2, 124836, 137791], 
           [1, 0, 0, 0, 7.7, 13, 1, 77012, 84536], 
           [1, 0, 0, 1, 7.3, 6, 1, 67277, 72721], 
           [1, 0, 0, 1, 7.1, 9, 1, 73448, 80294], 
           [2, 1, 1, 1, 5.1, 10, 1, 97707, 101617], 
           [2, 1, 1, 1, 5.1, 9, 3, 106130, 118258], 
           [2, 1, 0, 1, 6.8, 8, 3, 81049, 88505], 
           [2, 1, 1, 1, 5.5, 25, 3, 140315, 150710], 
           [2, 1, 0, 1, 7.1, 11, 3, 84571, 92879], 
           [2, 1, 0, 1, 6.6, 9, 2, 58126, 64522], 
           [2, 1, 1, 1, 6.1, 27, 3, 133284, 151373], 
           [2, 1, 0, 1, 7.5, 32, 2, 142333, 154372], 
           [2, 1, 0, 0, 7.3, 23, 3, 71073, 78715], 
           [2, 1, 0, 0, 7.5, 14, 2, 76255, 82466], 
           [2, 1, 0, 1, 7.8, 11, 3, 105348, 118635], 
           [2, 1, 0, 0, 7.5, 9, 3, 86844, 97061], 
           [2, 1, 0, 0, 7, 5, 2, 52582, 58923], 
           [2, 1, 0, 1, 7.5, 12, 3, 110836, 120397], 
           [2, 1, 1, 1, 5.7, 18, 3, 144150, 151887], 
           [2, 1, 1, 1, 6, 19, 3, 122240, 134882], 
           [2, 1, 0, 0, 7.1, 10, 3, 68573, 75286], 
           [2, 1, 0, 0, 7.9, 8, 2, 36946, 69526], 
           [2, 1, 1, 1, 5.9, 4, 1, 71404, 78589], 
           [2, 1, 0, 1, 7.6, 14, 3, 102076, 112009], 
           [2, 0, 1, 1, 5.3, 10, 2, 94715, 106608], 
           [2, 0, 0, 1, 8.1, 4, 1, 58634, 62601], 
           [2, 0, 0, 1, 7.9, 14, 3, 119851, 131050], 
           [2, 0, 1, 0, 5.5, 9, 3, 78651, 88661], 
           [2, 0, 0, 0, 6.9, 8, 2, 57862, 63728], 
           [2, 0, 1, 0, 4.9, 3, 1, 67113, 74272], 
           [2, 0, 1, 0, 5.3, 12, 3, 81078, 86221], 
           [2, 0, 0, 1, 8.5, 7, 1, 60403, 68828], 
           [2, 0, 0, 1, 7.6, 12, 2, 90964, 97303], 
           [2, 0, 0, 1, 8.4, 4, 1, 12945, 57841], 
           [2, 0, 0, 0, 7.4, 5, 1, 56550, 62323], 
           [2, 0, 0, 0, 7.2, 2, 1, 34514, 38675], 
           [2, 0, 0, 0, 8, 8, 2, 56177, 61652], 
           [2, 0, 1, 0, 5, 2, 1, 50588, 57431], 
           [2, 0, 0, 1, 7.7, 5, 1, 60415, 66323], 
           [2, 0, 1, 0, 6.1, 11, 3, 91409, 102511], 
           [2, 0, 0, 1, 7.8, 3, 1, 56250, 59530], 
           [2, 0, 1, 1, 5.7, 16, 2, 127725, 143348], 
           [2, 0, 1, 0, 5.4, 5, 1, 63426, 71068], 
           [2, 0, 0, 0, 8, 5, 1, 45826, 48138], 
           [3, 1, 0, 1, 6.7, 24, 3, 154326, 165884], 
           [3, 1, 0, 0, 5.9, 11, 2, 87351, 95429], 
           [3, 1, 1, 1, 4.1, 5, 1, 123859, 130292], 
           [3, 1, 1, 0, 4.8, 16, 3, 118121, 130697], 
           [3, 1, 1, 1, 4.3, 11, 3, 174479, 193342], 
           [3, 1, 1, 1, 4.3, 16, 3, 196704, 218955], 
           [3, 1, 1, 1, 4.3, 16, 3, 186953, 206984], 
           [3, 1, 1, 0, 4.2, 9, 2, 107862, 117240], 
           [3, 1, 1, 0, 3.9, 5, 1, 105821, 116932], 
           [3, 1, 1, 1, 5.6, 6, 1, 114730, 126118], 
           [3, 0, 1, 0, 3.9, 9, 3, 142382, 151698], 
           [3, 0, 0, 0, 7.3, 9, 1, 67581, 72896], 
           [3, 0, 0, 0, 6.8, 6, 1, 67934, 75659], 
           [3, 0, 1, 0, 5, 10, 3, 118101, 129862], 
           [3, 0, 1, 1, 5.2, 9, 2, 127710, 143799], 
           [3, 0, 0, 0, 6.5, 5, 1, 70794, 77113], 
           [3, 0, 1, 0, 5.7, 4, 1, 81728, 91764], 
           [3, 0, 0, 0, 7, 7, 1, 71015, 80747], 
           [3, 0, 0, 1, 7.3, 11, 2, 117276, 126139], 
           [3, 0, 0, 0, 7.2, 10, 2, 106535, 117574], 
           [3, 0, 0, 0, 8.4, 5, 1, 68825, 74209], 
           [4, 1, 1, 1, 3.1, 6, 1, 109599, 121829], 
           [4, 1, 0, 1, 5.2, 17, 3, 154159, 169191], 
           [4, 1, 1, 1, 3.3, 10, 2, 145503, 162253], 
           [4, 1, 1, 1, 3.2, 15, 2, 161383, 181196], 
           [4, 1, 0, 1, 4.8, 10, 3, 113443, 126189], 
           [4, 1, 1, 1, 2.1, 5, 1, 118561, 131704], 
           [4, 1, 1, 1, 2.9, 8, 2, 147709, 161814], 
           [4, 1, 1, 1, 3.1, 7, 2, 121969, 136415], 
           [4, 1, 0, 0, 5.2, 11, 2, 101890, 113893], 
           [4, 1, 1, 1, 4, 7, 1, 114233, 127955], 
           [4, 0, 1, 1, 4.2, 4, 1, 77087, 84890], 
           [4, 0, 0, 1, 5.5, 6, 1, 113382, 124153], 
           [4, 0, 1, 1, 2.7, 4, 1, 102177, 112844], 
           [4, 0, 1, 1, 3.1, 2, 2, 116506, 127815], 
           [4, 0, 1, 1, 3.1, 5, 1, 130286, 145073], 
           [4, 0, 1, 1, 3.6, 16, 3, 159687, 177706], 
           [4, 0, 1, 1, 3.4, 2, 1, 107750, 117214], 
           [4, 0, 0, 1, 5.3, 4, 1, 91211, 103513], 
           [4, 0, 1, 1, 3, 6, 2, 135216, 150489], 
           [4, 0, 1, 1, 3.2, 10, 2, 164048, 182049], 
           [4, 0, 1, 1, 3.8, 5, 1, 84390, 92512], 
           [4, 0, 1, 1, 3.1, 14, 3, 170428, 186726], 
           [4, 0, 1, 1, 2.6, 5, 1, 124994, 140026], 
           [4, 0, 0, 1, 5.7, 13, 1, 90130, 95908], 
           [4, 0, 0, 1, 6.3, 5, 1, 102021, 109758], 
           [4, 0, 1, 0, 4, 4, 2, 88484, 99505], 
           [4, 0, 1, 1, 3, 7, 1, 110975, 123139], 
           [4, 0, 0, 1, 5.5, 3, 1, 81422, 89202], 
           [4, 0, 1, 1, 3.1, 10, 1, 130939, 142566], 
           [4, 0, 1, 1, 3.7, 1, 1, 111630, 125268], 
           [5, 1, 1, 1, 3.7, 11, 1, 182945, 198762], 
           [5, 1, 1, 0, 3.2, 13, 2, 163671, 181774], 
           [5, 1, 1, 1, 4.3, 9, 2, 204433, 226581], 
           [5, 1, 1, 1, 3.2, 4, 1, 135139, 148117], 
           [5, 1, 0, 1, 5.6, 11, 3, 147213, 155826], 
           [5, 1, 1, 1, 4, 13, 2, 207737, 229706], 
           [5, 1, 1, 1, 3.1, 18, 3, 244025, 268210], 
           [5, 1, 0, 1, 5.2, 12, 3, 167199, 182401], 
           [5, 1, 1, 1, 4, 14, 3, 209377, 225498], 
           [5, 1, 1, 1, 3.7, 10, 2, 209189, 230270], 
           [5, 1, 1, 1, 3.2, 11, 2, 186973, 204769], 
           [5, 1, 1, 1, 3.1, 10, 3, 209171, 229301], 
           [5, 1, 1, 1, 4.5, 15, 3, 230268, 253090], 
           [5, 1, 1, 1, 4.4, 8, 2, 186022, 207634], 
           [5, 1, 1, 1, 4.2, 7, 1, 179897, 196338], 
           [5, 1, 1, 1, 3.6, 12, 2, 236470, 265121], 
           [5, 1, 0, 0, 5.7, 15, 3, 172793, 184391], 
           [5, 1, 1, 1, 3.7, 13, 3, 285301, 297397],
           [5, 1, 1, 1, 3.6, 8, 3, 200543, 219584], 
           [5, 1, 1, 1, 4.8, 6, 1, 152485, 1668170], 
           [5, 1, 1, 1, 4, 5, 1, 154644, 170967], 
           [5, 1, 1, 1, 3.1, 22, 3, 255654, 285939], 
           [5, 1, 1, 1, 3.5, 2, 1, 138896, 152284], 
           [5, 1, 1, 1, 3.1, 18, 3, 260631, 284841], 
           [5, 1, 1, 1, 3, 4, 1, 183972, 201691], 
           [5, 1, 1, 1, 3, 5, 1, 128027, 142963], 
           [5, 1, 1, 1, 4, 6, 1, 161815, 173837], 
           [5, 1, 0, 1, 5.9, 6, 1, 149011, 163795], 
           [5, 1, 1, 1, 3.3, 8, 2, 167305, 183834], 
           [5, 1, 1, 1, 2.7, 5, 1, 153549, 168707], 
           [5, 1, 1, 1, 3.4, 37, 3, 300975, 332119], 
           [5, 1, 1, 0, 2.4, 3, 1, 124442, 131361], 
           [5, 1, 1, 1, 3.8, 14, 2, 163486, 175271], 
           [5, 1, 1, 1, 3.2, 7, 1, 155006, 169118], 
           [5, 1, 0, 0, 5.1, 8, 3, 129499, 141661], 
           [5, 1, 1, 1, 3.5, 14, 2, 216391, 234992], 
           [5, 1, 1, 0, 2.9, 8, 2, 171016, 189308], 
           [5, 1, 1, 1, 3.5, 25, 3, 246763, 279255], 
           [5, 1, 1, 1, 3.4, 5, 1, 175396, 189904], 
           [5, 1, 0, 1, 5.1, 8, 2, 144542, 158513], 
           [5, 1, 0, 1, 4.7, 21, 3, 275377, 294336], 
           [5, 1, 1, 1, 4, 18, 3, 262090, 289778], 
           [5, 1, 0, 0, 5.1, 2, 1, 250000, 276163], 
           [5, 1, 0, 1, 4.6, 10, 2, 158243, 177587], 
           [5, 1, 1, 0, 3.6, 26, 3, 218341, 245432], 
           [5, 1, 1, 0, 3.3, 5, 1, 141044, 157509], 
           [5, 1, 1, 0, 3.9, 10, 2, 164174, 183538], 
           [5, 1, 1, 1, 3.3, 16, 3, 203152, 216873], 
           [5, 1, 1, 1, 3.6, 8, 1, 194784, 217478], 
           [5, 1, 1, 1, 3, 15, 3, 223040, 239253], 
           [5, 0, 1, 0, 4.2, 3, 1, 138635, 149539], 
           [5, 0, 1, 1, 4.1, 5, 1, 176028, 192799], 
           [5, 0, 1, 1, 3.9, 2, 21, 126343, 136988], 
           [5, 0, 1, 1, 3.6, 14, 3, 249687, 267554], 
           [5, 0, 1, 1, 3.8, 5, 1, 127525, 139017], 
           [5, 0, 1, 1, 3.1, 13, 3, 281058, 313400], 
           [5, 0, 1, 1, 3.6, 4, 1, 149631, 165420], 
           [5, 0, 1, 1, 3.8, 4, 1, 142249, 154386], 
           [5, 0, 1, 1, 4.7, 7, 1, 166863, 189852], 
           [5, 0, 1, 1, 2.9, 7, 1, 164080, 180937], 
           [5, 0, 1, 0, 2.9, 1, 1, 112221, 126149], 
           [5, 0, 1, 0, 3.8, 5, 1, 161352, 177997], 
           [5, 0, 1, 1, 2.8, 7, 1, 2.9818, 233774], 
           [5, 0, 0, 1, 5.1, 6, 1, 133097, 147327], 
           [5, 0, 1, 1, 3.7, 5, 1, 171391, 185895], 
           [5, 0, 1, 1, 3.7, 6, 1, 166482, 180608], 
           [5, 0, 1, 1, 4, 12, 1, 242614, 265856], 
           [5, 0, 1, 1, 3.8, 6, 1, 146396, 161031], 
           [5, 0, 1, 1, 3.8, 6, 1, 130153, 139887], 
           [5, 0, 1, 1, 3.7, 8, 2, 175791, 195678], 
           [5, 0, 1, 0, 3.4, 3, 1, 138775, 152231], 
           [5, 0, 1, 1, 4.3, 6, 1, 167844, 189487], 
           [5, 0, 1, 1, 3.9, 10, 2, 202231, 225599], 
           [5, 0, 1, 0, 3.9, 2, 1, 106802, 114961], 
           [5, 0, 0, 1, 5.8, 23, 3, 204279, 227790], 
           [5, 0, 1, 1, 4.5, 8, 3, 185413, 207462], 
           [5, 0, 0, 1, 4.6, 9, 2, 150258, 160253], 
           [5, 0, 1, 1, 4, 5, 1, 128433, 145718], 
           [5, 0, 0, 1, 5.4, 14, 1, 123940, 138566], 
           [5, 0, 0, 1, 5.4, 3, 1, 107524, 116283], 
           [6, 1, 1, 1, 2.7, 6, 1, 214219, 236305], 
           [6, 1, 1, 1, 2.1, 5, 1, 312966, 336960], 
           [6, 1, 1, 1, 1.9, 21, 2, 342834, 367284], 
           [6, 1, 1, 1, 2.1, 16, 3, 312892, 344801], 
           [6, 1, 1, 1, 2.5, 13, 3, 317399, 345204], 
           [6, 1, 1, 1, 2.4, 21, 3, 342662, 369712], 
           [6, 1, 1, 1, 2.5, 5, 2, 247953, 273737], 
           [6, 1, 1, 1, 3.2, 11, 3, 339533, 387205], 
           [6, 1, 1, 1, 2.4, 22, 3, 393408, 435572], 
           [6, 1, 1, 1, 2.5, 5, 1, 206097, 220899], 
           [6, 1, 1, 1, 2.4, 27, 3, 419129, 472589], 
           [6, 1, 1, 1, 1.9, 5, 1, 250667, 270417], 
           [6, 1, 1, 1, 2.9, 6, 1, 212780, 237094], 
           [6, 1, 1, 1, 2.7, 7, 1, 232735, 261236], 
           [6, 1, 0, 1, 4.4, 11, 2, 244940, 269554], 
           [6, 1, 1, 1, 2.2, 13, 2, 287433, 314926], 
           [6, 1, 1, 1, 2.8, 7, 1, 279661, 317179], 
           [6, 1, 1, 1, 2.7, 12, 3, 428876, 462342], 
           [6, 1, 1, 1, 2.4, 10, 2, 326460, 364139], 
           [6, 1, 1, 1, 3.2, 10, 3, 366150, 408898], 
           [6, 1, 1, 1, 2.93, 8, 3, 299582, 339637], 
           [6, 1, 1, 1, 2.1, 5, 2, 259576, 280168], 
           [6, 1, 1, 1, 1.9, 12, 1, 31675, 349294], 
           [6, 1, 1, 1, 2.5, 7, 1, 220374, 239614], 
           [6, 1, 1, 1, 1.3, 7, 1, 262328, 295619], 
           [6, 1, 1, 1, 2.5, 14, 1, 337885, 376651], 
           [6, 1, 1, 1, 2.9, 14, 2, 355982, 389994], 
           [6, 1, 1, 0, 3.1, 16, 2, 298359, 320315], 
           [6, 1, 1, 1, 3, 6, 1, 247576, 271773], 
           [6, 1, 1, 1, 3.3, 15, 1, 251079, 275246], 
           [6, 1, 1, 0, 2.4, 2, 1, 154310, 171664], 
           [6, 1, 1, 1, 3.2, 25, 3, 411219, 455468], 
           [6, 1, 1, 1, 3, 15, 1, 266880, 297902], 
           [6, 1, 1, 1, 2.9, 5, 1, 223736, 248014], 
           [6, 1, 1, 1, 2.7, 10, 3, 322698, 356961], 
           [6, 0, 1, 1, 2.9, 8, 2, 297174, 323699],
           [6, 0, 1, 1, 2.6, 9, 3, 308081, 339664], 
           [6, 0, 1, 1, 2.8, 8, 1, 211269, 241577], 
           [6, 0, 1, 1, 2.4, 2, 1, 210801, 233905], 
           [6, 0, 1, 1, 3.2, 5, 1, 244551, 265502]]

In [20]:
import numpy as np

In [21]:
np_datawag = np.array(datawag)

In [22]:
print(datawag)

[[1, 1, 0, 0, 7.4, 9, 3, 77836, 84612], [1, 1, 0, 0, 6.7, 10, 2, 69994, 78497], [1, 1, 0, 0, 8.1, 6, 1, 82872, 67756], [1, 1, 1, 1, 5.1, 27, 3, 155196, 173220], [1, 1, 0, 0, 7.0, 10, 3, 89268, 96099], [1, 1, 0, 1, 7.7, 10, 3, 79714, 87531], [1, 1, 0, 1, 7.3, 9, 2, 89781, 99972], [1, 1, 1, 1, 5.3, 11, 3, 151423, 166601], [1, 1, 0, 0, 7.7, 18, 3, 81271, 85437], [1, 1, 0, 0, 7.5, 9.0, 2, 73018, 91369], [1, 1, 0, 0, 7.8, 12, 2, 80214, 88676], [1, 1, 0, 1, 7.3, 8.0, 2, 110302, 116654], [1, 1, 1, 1, 5.4, 18, 3, 137106, 149222], [1, 1, 0, 0, 7.5, 15, 2, 77545, 85041], [1, 1, 0, 0, 6.8, 10, 2, 90223, 97640], [1, 1, 0, 0, 8.1, 9.0, 3, 81119, 91800], [1, 1, 0, 0, 7.3, 16, 3, 90178, 101828], [1, 1, 0, 1, 7.7, 9.0, 3, 105564, 117743], [1, 1, 0, 0, 7.1, 37, 2, 115804, 126989], [1, 1, 0, 0, 7.2, 14, 3, 113149, 124432], [1, 1, 0, 1, 7.6, 27, 3, 157510, 168673], [1, 1, 0, 0, 7.1, 14, 3, 97104, 104090], [1, 1, 0, 1, 7.3, 17, 3, 131815, 145843], [1, 1, 0, 1, 6.9, 23, 3, 134135, 151837], [1, 1, 0, 1, 7.4

In [23]:
np_datawag[:,7]

array([  7.78360000e+04,   6.99940000e+04,   8.28720000e+04,
         1.55196000e+05,   8.92680000e+04,   7.97140000e+04,
         8.97810000e+04,   1.51423000e+05,   8.12710000e+04,
         7.30180000e+04,   8.02140000e+04,   1.10302000e+05,
         1.37106000e+05,   7.75450000e+04,   9.02230000e+04,
         8.11190000e+04,   9.01780000e+04,   1.05564000e+05,
         1.15804000e+05,   1.13149000e+05,   1.57510000e+05,
         9.71040000e+04,   1.31815000e+05,   1.34135000e+05,
         9.21980000e+04,   9.93040000e+04,   8.21320000e+04,
         1.08770000e+05,   1.09775000e+05,   6.63750000e+04,
         5.78380000e+04,   9.06310000e+04,   6.09300000e+04,
         8.72310000e+04,   6.85820000e+04,   6.32800000e+04,
         1.06412000e+05,   7.53370000e+04,   1.43334000e+05,
         9.07710000e+04,   8.17690000e+04,   9.41030000e+04,
         1.02631000e+05,   1.09164000e+05,   6.37790000e+04,
         7.53620000e+04,   1.24836000e+05,   7.70120000e+04,
         6.72770000e+04,

In [24]:
round(7.78360000e+04)

77836

In [27]:
np.mean(np_datawag[:,5])

10.229885057471265

next we will test id the salaries of male and female are statisically significntly different from oneanother. Next is the case of 1994.

In [39]:
femwag94 = np.array([57838, 90631, 60930, 87231, 68582, 63280, 106412, 65337, 143334, 90771, 81769, 94103, 102631, 109164, 63779, 75362, 124836, 77012, 67277, 73448, 94715, 58634, 119851, 78650, 57862, 67113, 81078, 60403, 90964, 52945, 56550, 34514, 56177, 50588, 60415, 91409, 56250, 127725, 63426, 45826, 142382, 67581, 67934, 118101, 127710, 70794, 81728, 71015, 117276, 106535, 68825, 77087, 113382, 102177, 116506, 130386, 159687, 107750, 91211, 135216, 164048, 84390, 170428, 124994, 90130, 102021, 88484, 110975, 81422, 130939, 111630, 138635, 176028, 126343, 249687, 127525, 281058, 149631, 142249, 166863, 164080, 112221, 161352, 209818, 133097, 171391, 166482, 242614, 146396, 130153, 175791, 138775, 167844, 202231, 106802, 204279, 185413, 150258, 128433, 123940, 107524, 297174, 308081, 211269, 210801, 244551])
malwag94 = np.array([77836, 69994, 62872, 155196, 89268, 79714, 89781, 151423, 81271, 83018, 80214, 110302, 137106, 77545, 90223, 81197, 90178, 105564, 115804, 113149, 157510, 97104, 131815, 134135, 92198, 99304, 82132, 108770, 109775, 66375, 97707, 106130, 81049, 140315, 84571, 58126, 133284, 142333, 71073, 76255, 105348, 86844, 52582, 110836, 144150, 122240, 68573, 63936, 71404, 102076, 154326, 87351, 123859, 118121, 174479, 196704, 186953, 107862, 105821, 114730, 109599, 154159, 145503, 161383, 113443, 118561, 147709, 121969, 101890, 114233, 182945, 163671, 204433, 135139, 147213, 207737, 244025, 167199, 209377, 209189, 186973, 209171, 230268, 186022, 179897, 236470, 172793, 285301, 200543, 152485, 154644, 255654, 138896, 260631, 183972, 128027, 161815, 149011, 167305, 153549, 300975, 124442, 163486, 155006, 129499, 216391, 171016, 246763, 175396, 144542, 275377, 262090, 250000, 158243, 218341, 141044, 164174, 203152, 194784, 223040, 214219, 312966, 342834, 312892, 317399, 342662, 247953, 339533, 393408, 206097, 419129, 250667, 212780, 232735, 244940, 287433, 279661, 428876, 326460, 366150, 299582, 259576, 316754, 220374, 262328, 337885, 355982, 298359, 247576, 251079, 154310, 411219, 266880, 223736, 322698])

In [40]:
np_femwag94 = np.array(femwag94)

In [41]:
from scipy.stats import mannwhitneyu

In [42]:
u, p_value = mannwhitneyu(femwag94, malwag94)

In [43]:
print("two-sample wilcoxon-test", p_value)

two-sample wilcoxon-test 2.14071311034e-09


In [44]:
round(2.14071311034e-09)

0

Yes, it is statistically different

next we will test id the experience of male and female are statisically significntly different from oneanother.

In [56]:
femexp = np.array([2,
12,
7,
14,
8,
7,
16,
3,
16,
12,
8,
11,
10,
15,
9,
6,
13,
13,
6,
9,
10,
4,
14,
9,
8,
3,
12,
7,
12,
4,
5,
2,
8,
2,
5,
11,
3,
16,
5,
5,
9,
9,
6,
10,
9,
5,
4,
7,
11,
10,
5,
4,
6,
4,
2,
5,
16,
2,
4,
6,
10,
5,
14,
5,
13,
5,
4,
7,
3,
10,
1,
3,
5,
2,
14,
5,
13,
4,
4,
7,
7,
1,
5,
7,
6,
5,
6,
12,
6,
6,
8,
3,
6,
10,
2,
23,
8,
9,
5,
14,
3,
8,
9,
8, 
2, 
5])
malexp = np.array([9,
10,
6,
27,
10,
10,
9,
11,
18,
9,
12,
8,
18,
15,
10,
9,
16,
9,
37,
14,
27,
14,
17,
23,
8,
19,
15,
16,
15,
7,
10,
9,
8,
25,
11,
9,
27,
32,
23,
14,
11,
9,
5,
12,
18,
19,
10,
8,
4,
14,
24,
11,
5,
16,
11,
16,
16,
9,
5,
6,
6,
17,
10,
15,
10,
5,
8,
7,
11,
7,
11,
13,
9,
4,
11,
13,
18,
12,
14,
10,
11,
10,
15,
8,
7,
12,
15,
13,
8,
6,
5,
22,
2,
18,
4,
5,
6,
6,
8,
5,
37,
3,
14,
7,
8,
14,
8,
25,
5,
8,
21,
18,
2,
10,
26,
5,
10,
16,
8,
15,
6,
5,
21,
16,
13,
21,
5,
11,
22,
5,
27,
5,
6,
7,
11,
13,
7,
12,
10,
10,
8,
5,
12,
7,
7,
14,
14,
16,
6,
15,
2,
25,
15,
5,
10])

In [57]:
u, p_value = mannwhitneyu(femexp, malexp)

In [58]:
print("two-sample wilcoxon-test", p_value)

two-sample wilcoxon-test 4.96119877184e-10


In [60]:
round(4.96119877184e-10)

0

statistically different

In [59]:
np.mean(femexp)

7.4905660377358494

Average experience female

In [55]:
np.mean(malexp)

12.103225806451613

Average experience male
If we look at the average experience, we can conclude that the group with the stat sign larger experience is the male group.

In [66]:
import matplotlib.pyplot as plt

In [67]:
plt.bar(femexp, malexp, label='experience')

ValueError: shape mismatch: objects cannot be broadcast to a single shape

# Motivation 

In modern society, racism and discrimination are highly relevant topics. These topics are being discussed on countless occasions, given a wide variety of situations under which they occur. Recently, women spoke up about sexual harassment, while using #METOO, this mainly concerned the entertainment industry. Within the same industry, where you see a lot of stereotyping, it is also still the case that males have a significantly larger pay cut compared to females. Taking the initiative to discuss these topics has only been accepted and happening in recent years. This also goes for the topic of the increasing of gender equality. Not only with respect to the number of female managers, directors, politicians or other important job positions but also with respect to the equalization of salaries between male and female. This motivated us to analyze some data on salaries of males compared to females back in the day, when this was not a (daily) topic of discussion. 


# Method

# Results

# Conclusion